In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('https://raw.githubusercontent.com/farrelrassya/teachingMLDL/refs/heads/main/02.%20Deep%20Learning/Dataset/secondhanddataset.csv')
data.info()
data.isnull().sum()
data.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   v.id           1000 non-null   int64  
 1   on road old    1000 non-null   int64  
 2   on road now    1000 non-null   int64  
 3   years          1000 non-null   int64  
 4   km             1000 non-null   int64  
 5   rating         1000 non-null   int64  
 6   condition      1000 non-null   int64  
 7   economy        1000 non-null   int64  
 8   top speed      1000 non-null   int64  
 9   hp             1000 non-null   int64  
 10  torque         1000 non-null   int64  
 11  current price  1000 non-null   float64
dtypes: float64(1), int64(11)
memory usage: 93.9 KB


np.int64(0)

In [6]:
y = data['current price']
x = data.drop('current price', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape(-1, 1))
y_test = scaler.transform(y_test.reshape(-1, 1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [7]:
class NeuNet(nn.Module):
    def __init__(self, input):
        super(NeuNet, self).__init__()
        self.fc1 = nn.Linear(input, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = NeuNet(x_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

x_train_torch = torch.tensor(x_train, dtype=torch.float32)
x_test_torch = torch.tensor(x_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_torch = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

epochs = 101
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_out = model(x_test_torch)
        val_loss = criterion(val_out, y_test_torch)

    if epoch % 10 == 0:
        print(f'{epoch}, T.Loss: {loss.item()}, V.Loss: {val_loss.item()}')

y_pred_torch = model(x_test_torch).detach().numpy()
rmse_torch = np.sqrt(mean_squared_error(y_test, y_pred_torch))
mse_torch = mean_squared_error(y_test, y_pred_torch)
r2_torch = r2_score(y_test, y_pred_torch)
print(f'RMSE: {rmse_torch}, MSE: {mse_torch}, R2: {r2_torch}')


0, T.Loss: 1.0031635761260986, V.Loss: 0.9280834197998047
10, T.Loss: 0.9097673296928406, V.Loss: 0.8423687219619751
20, T.Loss: 0.7496910095214844, V.Loss: 0.6897793412208557
30, T.Loss: 0.5019429922103882, V.Loss: 0.4610300064086914
40, T.Loss: 0.21037453413009644, V.Loss: 0.19579745829105377
50, T.Loss: 0.043930720537900925, V.Loss: 0.047879770398139954
60, T.Loss: 0.03199850022792816, V.Loss: 0.03408706933259964
70, T.Loss: 0.014435154385864735, V.Loss: 0.021253766492009163
80, T.Loss: 0.010593832470476627, V.Loss: 0.015330586582422256
90, T.Loss: 0.007567812688648701, V.Loss: 0.011176283471286297
100, T.Loss: 0.006112869828939438, V.Loss: 0.009965009056031704
RMSE: 0.09982489163088763, MSE: 0.00996500898911846, R2: 0.9893353147910708


In [8]:
inputs = tf.keras.Input(shape=(x_train.shape[1],))
hid = tf.keras.layers.Dense(64, activation='relu')(inputs)
hid = tf.keras.layers.Dense(32, activation='relu')(hid)
hid = tf.keras.layers.Dense(16, activation='relu')(hid)
outputs = tf.keras.layers.Dense(1, activation='linear')(hid)

model = tf.keras.Model(inputs, outputs)
early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.compile(optimizer='adam', loss='mse')

history = model.fit(x_train, y_train, epochs=64, batch_size=32, validation_split=0.15, callbacks=[early])

y_pred_tf = model.predict(x_test)
rmse_tensor = np.sqrt(mean_squared_error(y_test, y_pred_tf))
mse_tensor = mean_squared_error(y_test, y_pred_tf)
r2_tensor = r2_score(y_test, y_pred_tf)
print(f'RMSE: {rmse_tensor}, MSE: {mse_tensor}, R2: {r2_tensor}')


Epoch 1/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.9395 - val_loss: 0.4650
Epoch 2/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3649 - val_loss: 0.1447
Epoch 3/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1007 - val_loss: 0.0750
Epoch 4/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0544 - val_loss: 0.0522
Epoch 5/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0364 - val_loss: 0.0402
Epoch 6/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0256 - val_loss: 0.0339
Epoch 7/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0197 - val_loss: 0.0298
Epoch 8/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0153 - val_loss: 0.0260
Epoch 9/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0124 - val_loss: 0.0246
Epoch 10/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0123 - val_loss: 0.0235
Epoch 11/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0087 - val_loss: 0.0225
Epoch 12/64
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0084 - va

In [9]:
import pandas as pd

data = {
    'Metric': ['RMSE', 'MSE', 'R2'],
    'PyTorch': [rmse_torch, mse_torch, r2_torch],
    'TensorFlow': [rmse_tensor, mse_tensor, r2_tensor]
}

df = pd.DataFrame(data)
df


,Metric,PyTorch,TensorFlow
0,RMSE,0.099825,0.117509
1,MSE,0.009965,0.013808
2,R2,0.989335,0.985222
